<a href="https://colab.research.google.com/github/K-BS/breathsound-use-stethoscope/blob/main/08_31_gridsearch_with_tabular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torchaudio
!pip install skorch

In [3]:
import pandas as pd
from pathlib import Path
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio
import torchvision
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from sklearn.metrics import confusion_matrix
import numpy as np
import sklearn.metrics as metrics
from torch.utils.data import Subset

In [4]:
Catholic_file = '/content/drive/MyDrive/breath_data/breath_v2.2/aug_train_v2.2.csv'
df = pd.read_csv(Catholic_file)
df

,filename,category,class,sex,old
0,0002-1.wav,non-wheezing,0,1,7
1,0002-2.wav,non-wheezing,0,1,7
2,0002-3.wav,non-wheezing,0,1,7
3,0002-4.wav,non-wheezing,0,1,7
4,0003-2.wav,non-wheezing,0,0,1
...,...,...,...,...,...
1605,wn_0535-4.wav,wheezing,1,0,9
1606,wn_0588-1.wav,wheezing,1,0,1
1607,wn_0599-3.wav,wheezing,1,1,6
1608,wn_0602-2.wav,wheezing,1,1,8


In [5]:
mean = df.old.mean()
std = df.old.std()
df.old = (df.old - mean) / std

In [6]:
df

,filename,category,class,sex,old
0,0002-1.wav,non-wheezing,0,1,0.905217
1,0002-2.wav,non-wheezing,0,1,0.905217
2,0002-3.wav,non-wheezing,0,1,0.905217
3,0002-4.wav,non-wheezing,0,1,0.905217
4,0003-2.wav,non-wheezing,0,0,-0.948195
...,...,...,...,...,...
1605,wn_0535-4.wav,wheezing,1,0,1.523021
1606,wn_0588-1.wav,wheezing,1,0,-0.948195
1607,wn_0599-3.wav,wheezing,1,1,0.596315
1608,wn_0602-2.wav,wheezing,1,1,1.214119


In [7]:
df['relative_path'] = '/' + df['filename'].astype(str)
df = df[['relative_path', 'class', 'sex', 'old']]
df.head()

,relative_path,class,sex,old
0,/0002-1.wav,0,1,0.905217
1,/0002-2.wav,0,1,0.905217
2,/0002-3.wav,0,1,0.905217
3,/0002-4.wav,0,1,0.905217
4,/0003-2.wav,0,0,-0.948195


In [8]:
data_path = '/content/drive/MyDrive/breath_data/breath_v2.2/aug_train_v2.2'

In [9]:
class Breath_sound_Util():
  
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    
    return (sig, sr)

  def resample(aud, newsr):
    sig, sr = aud
    
    if (sr == newsr):
     return aud

    num_channels = sig.shape[0]
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))
  

  def pad(aud, max_ms):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms
   
    if (sig_len > max_len):
      sig = sig[:,:max_len]
 
    elif (sig_len < max_len):

      repeated = []
      repeated.append(sig)
      required_len = max_len - sig_len

      while required_len > sig_len : 
        repeated.append(sig)
        require_len -= sig_len
      repeated.append(sig[:, :required_len])
 
      sig = torch.cat(repeated, 1)

    return (sig, sr)


  def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80
    
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)

  def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
    _, n_mels, n_steps = spec.shape
    mask_value = spec.mean()
    aug_spec = spec

    freq_mask_param = max_mask_pct * n_mels
    for _ in range(n_freq_masks):
      aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

    time_mask_param = max_mask_pct * n_steps
    for _ in range(n_time_masks):
      aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

    return aug_spec
    print(aug_spec)

In [10]:
class breathDS(Dataset):
    
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 48000
            
  def __len__(self):
    return len(self.df)    
    
  def __getitem__(self, idx):
    audio_file = self.data_path + self.df.loc[idx, 'relative_path']
    class_id = self.df.loc[idx, 'class']
    
    aud = Breath_sound_Util.open(audio_file)
    reaud = Breath_sound_Util.resample(aud, self.sr)
    dur_aud = Breath_sound_Util.pad(reaud, self.duration)
    sgram = Breath_sound_Util.spectro_gram(dur_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = Breath_sound_Util.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)
    #여기부터 나이, 성별변수를 집어넣는 과정임
    x = self.df.loc[idx, 'old']
    y = self.df.loc[idx, 'sex']
    x = torch.from_numpy(np.asarray(x).reshape((1,)))
    y = torch.from_numpy(np.asarray(y).reshape((1,)))
    tabular = torch.cat((x, y), 0)
    tabular = tabular.float()
    #print(tabular)
    #x1 = ["sex", "old"]
    #x2 = x2.
    #x2 = x2.iloc[idx].values
    
    
    return [aug_sgram, tabular], class_id

In [11]:
from skorch import NeuralNetClassifier

In [12]:
ds = breathDS(df, data_path)
#inputs = ds[0]
#inputs2 = ds[1]
#outputs = ds[1]

In [13]:
import numpy as np
from sklearn.datasets import make_classification
from torch import nn
import torch.nn.functional as F
import torch
from skorch import NeuralNetClassifier
from datetime import datetime, time as datetime_time, timedelta
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import GridSearchCV


class WhoWheezing(nn.Module):
    

    def __init__(self):
        super().__init__()
        conv_layers = []
        self.conv1 = nn.Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.dropout = nn.Dropout(0.4)
        self.lin = nn.Linear(in_features=72, out_features=2)
        self.dropout = nn.Dropout(0.4)
        self.sigmoid = nn.Sigmoid()

        self.conv = nn.Sequential(*conv_layers)
 
        self.fc1 = nn.Linear(2, 10)
        self.fc2 = nn.Linear(10, 8)

    def forward(self, input):
        inputs=input[0]
        inputs2=input[1]
        inputs = self.conv(inputs)
        inputs = self.ap(inputs)
        inputs = inputs.view(inputs.shape[0], -1)
        inputs2 = self.fc1(inputs2)
        inputs2 = self.fc2(inputs2)
        #inputs가 원래있던 멜스펙트로그램이고
        #inputs2가 정형데이터임
        x = torch.cat((inputs, inputs2), 1)
        x = self.lin(x)
        x = self.dropout(x)
        inputs = self.sigmoid(x)
        return x 

    '''def forward(self, x):
        x = self.conv(x)
        x = self.ap(x)
        x = x.view(x.shape[0], -1)
        #print(x.size())
        #x2 = category
        #x2 = x2(self.fc1(x2))
        #x2 = x2(self.fc2(x2))
        #x = x1 + x2
        x = self.lin(x)
        x = self.dropout(x)
        x = self.sigmoid(x)
        return x'''
    
Model1 = WhoWheezing()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Model1 = Model1.to(device)
next(Model1.parameters()).device
#input = ds[0]



params = {
    'lr': [0.0001, 0.001],
    'max_epochs': [40, 60, 80, 100],
    'batch_size': [16, 32, 64]
}

net = NeuralNetClassifier(
    Model1,
    max_epochs=50,
    lr=0.001,
    batch_size=16,
    iterator_train__shuffle=True,
    verbose=1,
    train_split=None
)

In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
print(ds[1][1])

0


In [16]:
from skorch.helper import SliceDataset
input = SliceDataset(ds, idx=0)
outputs = SliceDataset(ds, idx=1)

In [17]:
gs = GridSearchCV(net, params, refit=False, cv=5, scoring='accuracy', error_score="raise", verbose=3, n_jobs=-1)

In [18]:
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [ ]:
gs.fit(input, outputs)

In [ ]:
pd.DataFrame(gs.cv_results_)